In [2]:
import numpy as np
import random

# 初始化参数
def initialize_parameters(numIn, numHide, numOut):
    # w1和b1为输入层与隐藏层之间的权重和偏置，w2和b2为隐藏层与输出层之间的权重和偏置
    b1 = np.random.rand(numHide) *0.01
    b2 = np.random.rand(numOut) *0.01
    w1 = np.random.rand(numIn,numHide) *0.01
    w2 = np.random.rand(numHide,numOut) *0.01
    
    # 通过字典存储参数
    parameters = {'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2}
    return parameters

# 定义sigmoid函数
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

# 前向传播
def forward_propagation(x, parameters):
    # w1和b1为输入层与隐藏层之间的权重和偏置，w2和b2为隐藏层与输出层之间的权重和偏置
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']

    # 使用sigmoid函数作为激活函数，a1和z1为输入层与隐藏层之间的输入和输出，w2和b2为隐藏层与输出层之间的输入和输出
    a1 = np.dot(x,w1)-b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,w2)-b2 
    z2 = sigmoid(a2)

    cache = {'a1': a1, 'z1': z1, 'a2': a2, 'z2': z2}
    return z2,cache

# 计算损失
def loss(z2, y, m):
    # 采用交叉熵函数作为损失函数
    cost = - np.sum(np.multiply(np.log(z2), y) + np.multiply((1 - y), np.log(1 - z2))) / m
    return cost

# 反向传播
def backward_propagation(parameters, cache, x, y, n):
    w2 = parameters['w2']
    z1 = cache['z1']
    z2 = cache['z2']

    # 根据链式法则推导求导公式，计算e_w1、e_b1、e_w2、e_b2
    g = z2*(1-z2)*(y-z2) 
    e_w2 = np.dot(z1.T,g)
    e_b2 = -np.mean(g, axis=0)

    e = z1*(1-z1)*np.dot(g,w2.T)
    e_w1 = np.dot(x.T, e) 
    e_b1= -np.mean(e, axis=0)
    
    grads = {'e_w1': e_w1, 'e_b1': e_b1, 'e_w2': e_w2, 'e_b2': e_b2}
    return grads

# 更新参数
def update_parameters(parameters, grads, learning_rate):
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']
    e_w1 = grads['e_w1']
    e_b1 = grads['e_b1']
    e_w2 = grads['e_w2']
    e_b2 = grads['e_b2']

    w1 = w1 + e_w1 * learning_rate
    b1 = b1 + e_b1 * learning_rate
    w2 = w2 + e_w2 * learning_rate
    b2 = b2 + e_b2 * learning_rate

    parameters = {'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2}
    return parameters

# 测试函数
def test(parameters, x_test, y_test, m):
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']

    # 使用训练后得到的参数进行前向传播得到预测值
    a1 = np.dot(x_test,w1)-b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,w2)-b2 
    z2 = sigmoid(a2)
    
    # 对预测值分类
    output = np.zeros((m, 3), dtype=int)

    for i in range(m):
        for j in range(3):
            if z2[i][j] > 0.5:
                output[i][j] = 1
            else:
                output[i][j] = 0
                
    yuce=np.zeros(m,dtype=int)  
    
    for i in range(m):
        if output[i][0]==1 and output[i][1]==0 and output[i][2]==0:
            yuce[i]=1
        elif output[i][0]==0 and output[i][1]==1 and output[i][2]==0:
            yuce[i]=2
        elif output[i][0]==0 and output[i][1]==0 and output[i][2]==1:
            yuce[i]=3

    # 计算正确率
    right_num = 0
    for j in range(m):
        if yuce[j] == y_test[j] :
            right_num = right_num + 1
    right_rate = right_num / m 

    return right_rate

# 顶层模块，调用各函数组成BP神经网络模型
def neural_model(x, y, x_test, y_test, numIn, numHide, numOut, numIterations): 
    print('输入层结点数：%i  隐藏层结点数：%i  输出层结点数：%i  训练次数：%i' %(numIn,numHide,numOut,numIterations))
    n = x.shape[0]
    m = x_test.shape[0]
    # 初始化参数
    parameters = initialize_parameters(numIn, numHide, numOut)
    for i in range(numIterations):
        # 前向传播
        z2, cache = forward_propagation(x, parameters)
        # 计算损失
        cost = loss(z2, y, n)
        # 反向传播
        grads = backward_propagation(parameters, cache, x, y, n)
        # 更新参数
        parameters = update_parameters(parameters, grads, 0.001)
        # 进行测试
        
        # 每20次训练，打印输出一次损失和正确率
        if (i + 1) % 1000 == 0:         
            right_rate = test(parameters, x_test, y_test, m)
            print('训练%i次，损失为：%f  准确率：%.2f' % (i+1,cost,right_rate))
            
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']   
    print('训练后输入层与隐藏层之间的权值为：')
    print(w1)
    print('训练后输入层与隐藏层之间的偏置为：')
    print(b1.T)
    print('训练后隐藏层与输出层之间的权值为：')
    print(w2)
    print('训练后隐藏层与输出层之间的偏置为：')
    print(b2.T)


if __name__ == "__main__":
    # 读取数据
    x = []
    y = []
    result = []
    with open('wine.txt', 'r') as f:
        data = f.readlines()
    # 打乱数据顺序
    random.shuffle(data)
    # 读取参数x，结果y，值result
    for line in data:
        values = line.strip().split(',')
        x.append([float(v) for v in values[1:]])
        if values[0] == '1':
            y.append([1,0,0])
            result.append(1)
        elif values[0] == '2':
            y.append([0,1,0])
            result.append(2)
        elif values[0] == '3':
            y.append([0,0,1])
            result.append(3)

    x = np.array(x)
    y = np.array(y)
    result = np.array(result)
    # 对x元素进行处理
    x[:, 0] *= 0.1
    x[:, 12] *= 0.001    
    # 划分训练集和测试集
    x_train = x[:150]
    y_train = y[:150]
    x_test = x[150:]
    y_test = result[150:]

    # 使用BP神经网络模型进行训练，输入层13个结点，隐藏层40个结点，输出层3个结点，训练10000次
    parameters = neural_model(x, y, x_test, y_test, 13, 40, 3, 10000)
    

输入层结点数：13  隐藏层结点数：40  输出层结点数：3  训练次数：10000
训练1000次，损失为：1.528038  准确率：0.68
训练2000次，损失为：0.887053  准确率：0.86
训练3000次，损失为：0.598045  准确率：0.93
训练4000次，损失为：0.571902  准确率：0.93
训练5000次，损失为：0.452456  准确率：0.96
训练6000次，损失为：0.450502  准确率：0.89
训练7000次，损失为：0.646465  准确率：0.93
训练8000次，损失为：0.449911  准确率：0.89
训练9000次，损失为：0.390662  准确率：0.93
训练10000次，损失为：0.384657  准确率：0.93
训练后输入层与隐藏层之间的权值为：
[[ 6.82684652e-02  1.39954761e-03  3.25483813e-02  7.28897138e-02
   4.90873215e-03  5.03081057e-02  6.79849203e-03  1.52053972e-02
  -1.28874378e-01 -2.33558130e-03 -3.78618666e-02 -1.19710239e-02
   8.38456696e-03 -4.08479931e-02 -4.46852328e-02 -5.45757509e-02
   7.36369125e-02  8.91131289e-02 -3.94667955e-02 -4.61436276e-02
  -3.27233623e-02  8.50821438e-02  9.63874551e-02  5.50362590e-03
  -1.63118553e-01 -5.12665599e-02  9.99739791e-02 -7.85051275e-02
  -6.38894270e-02  6.12697803e-02  7.00789315e-02 -3.09789129e-02
   2.29646632e-02 -3.02123810e-02 -6.32001952e-02  6.61359545e-02
   6.92970184e-02  8.40663904e-02 